In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv('/content/drive/My Drive/DL Projects/lyrics-generator/lyrics.csv', encoding = 'latin')
dataset.head(5)

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."


In [3]:
dataset.shape

(362237, 6)

In [4]:
dataset['lyrics'][0].split('\n')

['Oh baby, how you doing?',
 "You know I'm gonna cut right to the chase",
 'Some women were made but me, myself',
 'I like to think that I was created for a special purpose',
 "You know, what's more special than you? You feel me",
 "It's on baby, let's get lost",
 "You don't need to call into work 'cause you're the boss",
 'For real, want you to show me how you feel',
 "I consider myself lucky, that's a big deal",
 'Why? Well, you got the key to my heart',
 "But you ain't gonna need it, I'd rather you open up my body",
 "And show me secrets, you didn't know was inside",
 'No need for me to lie',
 "It's too big, it's too wide",
 "It's too strong, it won't fit",
 "It's too much, it's too tough",
 "He talk like this 'cause he can back it up",
 'He got a big ego, such a huge ego',
 "I love his big ego, it's too much",
 "He walk like this 'cause he can back it up",
 "Usually I'm humble, right now I don't choose",
 'You can leave with me or you could have the blues',
 'Some call it arrogant,

In [0]:
# Save Lyrics in .txt file
with open('/content/drive/My Drive/DL Projects/lyrics-generator/lyricsText.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in dataset['lyrics'][:10000]:
        filehandle.write('%s\n' % listitem)

In [0]:
# Load the dataset and convert it to lowercase :
textFileName = '/content/drive/My Drive/DL Projects/lyrics-generator/lyricsText.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()
raw_text = raw_text.lower()

In [7]:
chars = sorted(list(set(raw_text)))
print(chars[10:70])
int_to_chars = dict((i,c) for i, c in enumerate(chars))
chars_to_int = dict((i,c) for c, i in enumerate(chars))

['$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']


In [8]:
n_chars = len(raw_text)
n_vocabs = len(chars) #unique characters
print(n_chars)
print(n_vocabs)

8088566
143


In [9]:
from tqdm import tqdm
seq_len = 20
data_X = []
data_Y = []

for i in tqdm(range(0, n_chars - seq_len, 1)):
    seq_in = raw_text[i: i+seq_len]
    seq_out = raw_text[i + seq_len]
    data_X.append([chars_to_int[char] for char in seq_in])
    data_Y.append([chars_to_int[seq_out] for char in seq_out])

n_patterns = len(data_X)
print('total patterns : ', n_patterns)

100%|██████████| 8088546/8088546 [00:32<00:00, 252208.10it/s]

total patterns :  8088546


In [0]:
import sys 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [0]:


# comment the above lines if you uncomment the above cells

X = np.reshape(data_X, (n_patterns, seq_len, 1))

X = X / float(n_vocabs)



In [0]:
y = np_utils.to_categorical(data_Y)

In [0]:
LSTM_layer_num = 3 # number of LSTM layers
layer_size = [10,10,10] # number of nodes in each layer

In [0]:
model = Sequential()

In [0]:
model.add(LSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))

In [0]:
for i in range(1,LSTM_layer_num) :
    model.add(LSTM(layer_size[i], return_sequences=True))
    model.add(Dropout(0.1))

In [0]:
model.add(Flatten())

In [0]:
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 10)            480       
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 10)            840       
_________________________________________________________________
dropout (Dropout)            (None, 20, 10)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 10)            840       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 10)            0         
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 143)               2

In [0]:
# Configure the checkpoint :
checkpoint_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

In [0]:
import tensorflow
es = tensorflow.keras.callbacks.EarlyStopping(monitor='loss', mode='min', baseline=0.4)


In [42]:
# Fit the model :

model.fit(X, y, epochs = 20, batch_size = 512, callbacks=[es]) 

Epoch 1/20
15798/15798 [==============================] - 125s 8ms/step - loss: 2.4100 - accuracy: 0.3047


In [46]:
# set a random seed :
start = np.random.randint(0, len(data_X)-1)
pattern = data_X[start]
print('Seed : ')
print("\"",''.join([int_to_chars[value] for value in pattern]), "\"\n")
# How many characters you want to generate
generated_characters = 300
# Generate Charachters :
for i in range(generated_characters):
    x = np.reshape(pattern, ( 1, len(pattern), 1))
    x = x / float(n_vocabs)
    prediction = model.predict(x,verbose = 0)
    index = np.argmax(prediction)
    result = int_to_chars[index]
    #seq_in = [int_chars[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('\nDone')

Seed : 
" on warm afternoons,
 "

the soeet the so the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the the the so the th
Done
